In [ ]:
# 📄 PDF 문서
#    ↓ (Upstage DocumentParser)
# 📜 전체 텍스트
#    ↓ (페이지 단위 청킹)
# 🧩 context 청크들
#    ↓ (GPT-4 API 호출, 커스텀 프롬프트로 질문/정답/이유 생성/CoT Reasoning)
# 📦 question / context / reason & answer /
#    ↓
# 💾 QA datasets csv 저장 (RAFT fine-tuning에 사용)

In [70]:
from dotenv import load_dotenv

load_dotenv()

True

## RAFT 학습용 QA 생성기

In [ ]:
import re
import os
import requests
import json
from tqdm import tqdm
import openai
from langchain_core.documents import Document

In [74]:
# ✅ 파일 경로 설정
file_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Korean_guideline_of_pediatric_procedural_sedation.pdf"
file_name = os.path.basename(file_path)
output_path = os.path.join(os.path.dirname(file_path), f"{os.path.splitext(file_name)[0]}.json")

In [ ]:
import os
import requests
import json

# ✅ 기본 요청 설정
DEFAULT_CONFIG = {
    "ocr": False,
    "coordinates": True,
    "output_formats": "['html', 'text', 'markdown']", 
    "model": "document-parse",
    "base64_encoding": "['figure', 'chart', 'table']" 
}

# ✅ Upstage API 설정
api_url = "https://api.upstage.ai/v1/document-ai/document-parse"
api_key = os.environ.get("UPSTAGE_API_KEY")
headers = {"Authorization": f"Bearer {api_key}"}

# ✅ 분석 요청 실행
print(f"📤 파일 업로드 중: {file_name}")
with open(file_path, "rb") as pdf:
    response = requests.post(
        api_url,
        headers=headers,
        data=DEFAULT_CONFIG,
        files={"document": pdf}
    )

# ✅ 결과 처리 및 metadata 추가 저장
if response.status_code == 200:
    result = response.json()
    
    # 📎 메타데이터 구성
    metadata = {
        "document_name": file_name,
        "model": result.get("model"),
        "usage": result.get("usage"),
        "num_pages": len({elem["page"] for elem in result.get("elements", [])}),
    }

    # 결과와 메타데이터를 함께 저장
    result_with_metadata = {
        "metadata": metadata,
        "elements": result.get("elements", [])
    }

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(result_with_metadata, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 분석 결과 및 메타데이터 저장 완료: {output_path}")
else:
    print(f"❌ 오류 발생 ({response.status_code}): {response.text}")


📤 파일 업로드 중: Korean_guideline_of_pediatric_procedural_sedation.pdf
✅ 분석 결과 및 메타데이터 저장 완료: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Korean_guideline_of_pediatric_procedural_sedation.json


In [106]:
# ✅ 분석 결과 JSON 파일 경로
json_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Korean_guideline_of_pediatric_procedural_sedation.json"


In [108]:
# ✅ 분석 결과 JSON 파일 경로
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

elements = data["elements"]

# ✅ 페이지 필터링 + 문단 추출
remove_pages = set(range(0, 5)) | set(range(63, 68))

filtered_elements = [
    elem for elem in elements
    if elem["category"] not in ("footer", "header", "footnote")
    and elem["page"] not in remove_pages
    and elem["content"].get("markdown", "").strip()
]

In [116]:
# 제거할 페이지 필터링 + 문단 결합
from langchain_core.documents import Document

remove_pages = set(range(0, 35)) | set(range(57, 69))  # 필요 없는 페이지 번호 제거
page_map = {}
for elem in elements:
    page = elem["page"]
    if page in remove_pages or elem["category"] in ("footer", "header", "footnote"):
        continue
    text = elem["content"].get("markdown", "").strip()
    if text:
        page_map[page] = page_map.get(page, "") + "\n" + text

In [117]:
#LangChain Document 구성
documents = [
    Document(
        page_content=page_map[page].strip(),
        metadata={"page": page, "document_name": file_name}
    )
    for page in sorted(page_map.keys())
]

print(f"✅ 구성된 문서 수: {len(documents)}")


✅ 구성된 문서 수: 22


In [118]:
# split_docs는 페이지 단위로 그대로 사용
split_docs = documents
print(f"✅ QA 생성을 위한 페이지 청크 수: {len(split_docs)}")

✅ QA 생성을 위한 페이지 청크 수: 22


In [119]:
split_docs

[Document(metadata={'page': 35, 'document_name': 'Korean_guideline_of_pediatric_procedural_sedation.pdf'}, page_content='# 1. 약물 투여 경로\n# 1) 경구 투여\n가장 손쉽게 이용할 수 있고 주사를 피함으로써 약물 복용의 순응도를 높일 수 있어 가장 많이\n사용되는 경로이다. 경미한 행동 장애가 있는 소아에서도 유용한 방법이다. 그러나 약물이 위장\n관에서 흡수되는 정도가 개인마다 다양하며 간을 통과할 때 일차 통과 대사(first-pass\nmetabolism)가 일어나 진정의 효과를 각 환자마다 정확하게 예측하는 것이 어려운 단점이 있다.\n# 2) 흡입 투여\n투여가 간편하고 주사를 놓지 않아 환자의 순응도를 높일 수 있다. 흡입된 마취제가 폐포를 통\n해 흡수되어 폐순환계를 거쳐 바로 전신순환계로 이동하므로 작용시간이 매우 빠르다. 따라서\n투여된 흡입마취제의 효과를 적정하는 것이 손쉽고, 일정 수준의 진정 심도를 유지하는 것이\n가능하다. 하지만 흡입 진정에 주로 사용되는 아산화질소(N2O)의 경우 단독 사용 시 마취 효과\n가 낮아서 깊은 진정을 유지하는 것이 어렵고, 확산성 저산소증의 위험이 있기 때문에 진정 전\n후의 산소 투여가 필요하다.\n# 3) 비강내 투여\n경구 투여를 거부하는 매우 비협조적 환자에서 사용이 가능하다. 비강 내로 투여한 약물이 비\n점막을 통해 흡수되어 곧바로 전신 효과를 나타내는 것으로 생각되며, 장관 순환을 거치지 않\n아 흡수 속도가 경구 투여보다 더 빠르다. 주로 투여 10-15분 정도에 임상 효과가 나타나는 것\n으로 알려져 있다.\n# 4) 근육내 투여\n비경구적 투여 방법 중 많이 이용되는 방법으로 흡수된 약물이 간-장관계 순환을 거치지 않아\n일차 통과 대사 없이 직접 심혈관계로 들어가기 때문에, 경구 진정에 비해 약물 흡수 양을 예\n측할 수 있고 더 빠른 발현을 보인다. 협조가 불가능한 환자에서 매우 유

In [ ]:
# from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnableLambda
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# import json
# import difflib

# prompt = PromptTemplate.from_template(
#     """Context information is below. You are only aware of this context and nothing else.

# ### Context:
# ---------------------
# {context}
# ---------------------

# You are a medical expert in the field of {domain}. Based on the above clinical guideline context, your task is to generate exactly {num_questions} question(s) that a medical professional might ask to assess clinical understanding.

# ---

# Each Q&A pair must:
# - Present a **clinically plausible and distinct** question (no duplicates or variations of the same question)
# - Be grounded **explicitly in the given context** (no hallucinations)
# - Belong to one of the following clinical reasoning categories:
#   • Drug selection & dosing (e.g., appropriate drug, route, weight-based dosing)
#   • Contraindications & precautions
#   • Sedation monitoring & depth assessment
#   • Pre-sedation evaluation
#   • Recovery & discharge criteria
#   • Emergency management

# ---

# To add clinical realism, consider:
# - Comorbidities
# - Risk-benefit tradeoffs
# - Dosing adjustments or monitoring strategies

# ---

# Each entry must include:
# - `question`: A Korean clinical question
# - `reason`: A Korean explanation citing direct quotes using ##begin_quote## and ##end_quote##
# - `answer`: A Korean answer based strictly on the context

# To ensure quality:
# - Do not repeat medications, topics, or sentence structures
# - Vary the structure of your reasoning to avoid templated style

# Respond only in **valid JSON array format**. No markdown, no lists, no extra explanations.

# ---

# ### Example Format:
# [
#   {{
#     "question": "4살 환자의 적절한 endotracheal tube size와 depth는?",
#     "reason": "문서에 따르면 ##begin_quote## ETT 크기(직경, mm) = (나이/4) + 4 / 깊이(cm) = (나이/2) + 12 ##end_quote## 라고 명시되어 있습니다.",
#     "answer": "비커프 튜브는 5.0 mm, 커프 튜브는 4.5 mm, 깊이는 약 14 cm가 적절합니다."
#   }},
#   {{
#     "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떻게 해야 하나요?",
#     "reason": "문서에서는 ##begin_quote## 후두경련 발생 시 양압환기, 하악 자극, 약물 투여, 기도 확보가 필요하다 ##end_quote## 라고 명시되어 있습니다.",
#     "answer": "산소 양압환기, 하악 자극, 프로포폴 또는 석시닐콜린 투여, 필요시 재삽관을 고려합니다."
#   }}
# ]
# """
# )


In [113]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from tqdm import tqdm
import difflib
import json

# ✅ 프롬프트: 질문 + reference + Reasoning 포함
prompt = PromptTemplate.from_template(
    """You are a medical expert in the field of {domain}, especially trained in pediatric sedation.

Your task is to generate **Korean Q&A data** from the guideline content provided below. This data will be used to fine-tune a medical question-answering model for pediatric anesthesiologists in Korea.

--- 
### Context:
 ---------------------
{context}
 ---------------------

Given the following guideline excerpt, generate exactly {num_questions} items. Each item must include:

# Guidelines for Question Generation
1. **Always generate questions strictly based on the content of the given page.**
2. Focus on questions that resemble those a practicing pediatric anesthesiologist might ask in real clinical settings.
3. Avoid multiple-choice formats. All questions must be open-ended, short-answer (descriptive) questions.
4. Include specific numerical values in the question whenever available (e.g., doses, weights, ages, volumes, concentrations).
5. Avoid general or overly broad questions. Be as specific and practical as possible.
6. **The question must be written in Korean.**

# Guidelines for Reference Sentences
- Return all sentences that clearly support the generated question.
- If the question is derived from a table, return the **entire table** textually.
- If the question is based on a figure or image, include the **sentences before and after the figure** along with its caption, if any.


Only use the given context.

Return your output in this **strict JSON format**:
[
  {{
  "question": "<your generated question here in Korean>",
  "reference_sentences": [
    "<sentence 1>",
    "<sentence 2>",
    ...
  ],
    "answer": "##Reason: ...\n<ANSWER>: ..."
  }}
]

--- 

### Answer Guidelines:
- Start with a step-by-step reasoning using quotes from the context
- Enclose all direct quotes in **##begin_quote## ... ##end_quote##**
- End with `<ANSWER>: ...` in Korean (full sentence)

---

### Example Outputs
Return as a **valid JSON array** of objects. Each object must look like:

[
  {{
    "question": "5살 23kg 소아 환자의 마취 중 적절한 수액 주입량은?",
    "reference_sentences": [
      "20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다.",
      "이 수액량은 전신마취 중 적절한 수분 공급을 위해 사용된다."
    ],
    "answer": "##Reason: 문서의 ##begin_quote## 20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다 ##end_quote## 라는 설명에 따르면, 23kg 소아는 1500mL + (3×20mL) = 1560mL가 필요합니다.\n<ANSWER>: 5살 23kg 소아의 마취 중 유지 수액량은 1560mL입니다."
  }},
  {{
    "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떤 처치를 해야 하나요?",
    "reference_sentences": [
      "Laryngospasm이 의심되는 경우 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다.",
      "신속한 인식과 처치가 저산소증을 예방하는 데 중요하다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다 ##end_quote## 라고 되어 있습니다. 이는 환자의 기도를 유지하고 저산소증을 방지하는 데 중요한 조치입니다.\n<ANSWER>: Laryngospasm이 의심되는 경우 jaw thrust, 양압 환기, succinylcholine 투여를 포함한 즉각적인 처치가 필요합니다."
  }}
]
"""
)


In [114]:
# ✅ 2. LLM 설정
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# ✅ 3. 응답을 JSON 배열로 파싱
def custom_json_parser(response):
    try:
        json_string = response.content.strip().removeprefix("```json\n").removesuffix("\n```").strip()
        json_string = f'[{json_string}]' if not json_string.startswith("[") else json_string
        return json.loads(json_string)
    except Exception as e:
        print(f"[PARSE ERROR] {e}")
        return []

In [115]:
# ✅ 4. 체인 구성
raft_chain = prompt | llm | RunnableLambda(custom_json_parser)

In [ ]:
from tqdm import tqdm
import difflib
import re
import json
import os

# ✅ 설정
qa_dataset = []
question_set = set()
max_qa_count = 60
output_path = "qa_dataset.jsonl"
log_invalid_output = "qa_invalid_outputs.jsonl"

# ✅ 파일 초기화
open(output_path, "w", encoding="utf-8").close()
open(log_invalid_output, "w", encoding="utf-8").close()

# ✅ 진행률 표시
progress = tqdm(total=max_qa_count)
progress.set_description(f"생성된 QA: 0 / {max_qa_count}")

doc_idx = 0
while len(qa_dataset) < max_qa_count and doc_idx < len(split_docs):
    doc = split_docs[doc_idx]
    context = doc.page_content
    metadata = doc.metadata or {}
    remaining = max_qa_count - len(qa_dataset)

    # ✅ context 유효성 검사
    if not context.strip() or not re.search(r"\w", context):
        print(f"[SKIPPED: empty or non-text context] page={metadata.get('page')}")
        doc_idx += 1
        continue

    # ✅ 다양성 확보 (prompt sampling)
    llm.temperature = 0.2 if doc_idx % 2 == 0 else 0.5

    # ✅ GPT 호출
    results = raft_chain.invoke({
        "context": context,
        "domain": "Pediatric Anesthesia",
        "num_questions": min(2, remaining)
    })

    for qa in results:
        question = qa.get("question", "").strip()
        answer = qa.get("answer", "").strip()
        references = qa.get("reference_sentences", [])

        # ✅ 1. 중복 질문 필터링
        if not question or any(difflib.SequenceMatcher(None, question, q).ratio() > 0.85 for q in question_set):
            continue

        # ✅ 2. 정답 포맷 검사
        if not re.match(r"^##Reason: .*<ANSWER>: .*", answer, re.DOTALL):
            print(f"[SKIPPED: answer format invalid] {question}")
            with open(log_invalid_output, "a", encoding="utf-8") as f:
                json.dump({"issue": "invalid_answer_format", "question": question, "answer": answer}, f, ensure_ascii=False)
                f.write("\n")
            continue

        # # ✅ 3. reference 문장 유효성 검사
        # if not all(ref in context for ref in references):
        #     print(f"[SKIPPED: reference mismatch] {question}")
        #     with open(log_invalid_output, "a", encoding="utf-8") as f:
        #         json.dump({"issue": "invalid_reference", "question": question, "references": references}, f, ensure_ascii=False)
        #         f.write("\n")
        #     continue

        # # ✅ 4. 질문 필터링 (너무 짧거나 의미 없음)
        # if len(question) < 10 or not any(kw in question for kw in ["환자", "소아", "마취", "용량", "기준", "평가", "처치"]):
        #     print(f"[SKIPPED: trivial question] {question}")
        #     with open(log_invalid_output, "a", encoding="utf-8") as f:
        #         json.dump({"issue": "trivial_question", "question": question}, f, ensure_ascii=False)
        #         f.write("\n")
        #     continue

        # ✅ 저장
        question_set.add(question)
        qa["metadata"] = metadata
        qa_dataset.append(qa)

        with open(output_path, "a", encoding="utf-8") as f:
            f.write(json.dumps(qa, ensure_ascii=False) + "\n")

        progress.update(1)
        progress.set_description(f"생성된 QA: {len(qa_dataset)} / {max_qa_count}")

        if len(qa_dataset) >= max_qa_count:
            break

    doc_idx += 1

progress.close()
print(f"\n✅ 최종 생성된 QA 수: {len(qa_dataset)}")


In [120]:
## 35~56p까지만.

import re
import difflib
import json
from tqdm import tqdm

# ✅ 설정
qa_dataset = []
question_set = set()
output_path = "qa_dataset.jsonl"
invalid_log_path = "qa_invalid_outputs.jsonl"

# ✅ 초기화
open(output_path, "w", encoding="utf-8").close()
open(invalid_log_path, "w", encoding="utf-8").close()

# ✅ 진행률 표시
progress = tqdm(total=len(split_docs))
progress.set_description("QA 생성 중")

for doc_idx, doc in enumerate(split_docs):
    context = doc.page_content.strip()
    metadata = doc.metadata or {}

    if not context or len(context) < 100:
        print(f"[SKIPPED: empty/short context] page={metadata.get('page')}")
        progress.update(1)
        continue

    llm.temperature = 0.2 if doc_idx % 2 == 0 else 0.5

    try:
        results = raft_chain.invoke({
            "context": context,
            "domain": "Pediatric Anesthesia",
            "num_questions": 5  # 한 페이지당 최대 5개 요청
        })
    except Exception as e:
        print(f"[ERROR: GPT 호출 실패] page={metadata.get('page')}, error={e}")
        progress.update(1)
        continue

    valid_count = 0
    for qa in results:
        question = qa.get("question", "").strip()
        answer = qa.get("answer", "").strip()
        references = qa.get("reference_sentences", [])

        if not question or any(difflib.SequenceMatcher(None, question, q).ratio() > 0.85 for q in question_set):
            continue

        if not re.match(r"^##Reason: .*<ANSWER>: .*", answer, re.DOTALL):
            print(f"[SKIPPED: answer format invalid] {question}")
            with open(invalid_log_path, "a", encoding="utf-8") as f:
                json.dump({"issue": "invalid_answer_format", "question": question, "answer": answer}, f, ensure_ascii=False)
                f.write("\n")
            continue

        # 저장
        question_set.add(question)
        qa["metadata"] = metadata
        qa_dataset.append(qa)

        with open(output_path, "a", encoding="utf-8") as f:
            f.write(json.dumps(qa, ensure_ascii=False) + "\n")

        valid_count += 1
        if valid_count >= 5:  # ✅ 한 페이지당 최대 5개만 저장
            break

    progress.update(1)

progress.close()
print(f"\n✅ 생성 완료! 총 {len(qa_dataset)}개의 QA 생성됨.")


QA 생성 중:   0%|          | 0/22 [00:00<?, ?it/s]

```json
[
  {
    "question": "경구 투여의 단점은 무엇인가요?",
    "reference_sentences": [
      "가장 손쉽게 이용할 수 있고 주사를 피함으로써 약물 복용의 순응도를 높일 수 있어 가장 많이 사용되는 경로이다.",
      "그러나 약물이 위장관에서 흡수되는 정도가 개인마다 다양하며 간을 통과할 때 일차 통과 대사(first-pass metabolism)가 일어나 진정의 효과를 각 환자마다 정확하게 예측하는 것이 어려운 단점이 있다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 약물이 위장관에서 흡수되는 정도가 개인마다 다양하며 간을 통과할 때 일차 통과 대사(first-pass metabolism)가 일어나 진정의 효과를 각 환자마다 정확하게 예측하는 것이 어려운 단점이 있다 ##end_quote## 라고 설명되어 있습니다.
<ANSWER>: 경구 투여의 단점은 약물 흡수의 개인차와 일차 통과 대사로 인해 진정 효과를 예측하기 어렵다는 점입니다."
  },
  {
    "question": "흡입 투여의 장점은 무엇인가요?",
    "reference_sentences": [
      "투여가 간편하고 주사를 놓지 않아 환자의 순응도를 높일 수 있다.",
      "흡입된 마취제가 폐포를 통해 흡수되어 폐순환계를 거쳐 바로 전신순환계로 이동하므로 작용시간이 매우 빠르다.",
      "따라서 투여된 흡입마취제의 효과를 적정하는 것이 손쉽고, 일정 수준의 진정 심도를 유지하는 것이 가능하다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 투여가 간편하고 주사를 놓지 않아 환자의 순응도를 높일 수 있다 ##end_quote##, ##begin_quote## 흡입된 마취제가 폐포를 통해 흡수되어 폐순환계를 거쳐 바로 전신순환계로 이동하므로 작용시간이 매우 빠르다 ##end_quote##

QA 생성 중:   5%|▍         | 1/22 [00:12<04:17, 12.28s/it]

```json
[
  {
    "question": "Chloral hydrate의 최대 투여 용량은 얼마인가요?",
    "reference_sentences": [
      "| 진정 단계 | 진정 단계 | 진정 단계 | 진정 약물 | 용량 |",
      "| --- | --- | --- | --- | --- |",
      "| 최소 진정 | 중등도 진정 | 깊은 진정 | Drug | Dosage |",
      "| V | V | V | Chloral hydrate (pocral syrup) | · 경구: 25~100 mg/kg (시술 30-60 분 전) → 추가 용량 투여 30분 후 반복 투여 (최대 용량: 120 mg/kg 또는 2 g) |"
    ],
    "answer": "##Reason: 문서의 ##begin_quote## Chloral hydrate (pocral syrup) · 경구: 25~100 mg/kg (시술 30-60 분 전) → 추가 용량 투여 30분 후 반복 투여 (최대 용량: 120 mg/kg 또는 2 g) ##end_quote## 라는 설명에 따르면, Chloral hydrate의 최대 투여 용량은 120 mg/kg 또는 2 g입니다. <ANSWER>: Chloral hydrate의 최대 투여 용량은 120 mg/kg 또는 2 g입니다."
  },
  {
    "question": "Midazolam을 경구로 투여할 때 최대 용량은 얼마인가요?",
    "reference_sentences": [
      "| 진정 단계 | 진정 단계 | 진정 단계 | 진정 약물 | 용량 |",
      "| --- | --- | --- | --- | --- |",
      "|  | V | V | Midazolam | · 경구: 0.25~1 mg/kg (최대 용량 15 mg) · 근주: 0.1~0.3 mg/kg · 정주: 0.05~0.1 mg/kg · 비강: 0.2~0.5 mg/kg (최대 용량 15 mg) · 항

QA 생성 중:   9%|▉         | 2/22 [00:24<04:01, 12.07s/it]

...

```json
[
  {
    "question": "소아 환자에서 서맥이 발생했을 때 심박동수가 몇 회/분 미만일 경우 심폐소생술을 시행해야 하나요?",
    "reference_sentences": [
      "③ 산소와 환기가 유지되어도 순환이 원활하지 않으면서(저혈압, 의식저하) 심박동수가 60회/분 미만일 경우 심폐소생술을 시행한다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 산소와 환기가 유지되어도 순환이 원활하지 않으면서(저혈압, 의식저하) 심박동수가 60회/분 미만일 경우 심폐소생술을 시행한다 ##end_quote## 라고 명시되어 있습니다. 이는 심박동수가 60회/분 미만일 때 심폐소생술을 시작해야 함을 의미합니다. <ANSWER>: 소아 환자에서 심박동수가 60회/분 미만일 경우 심폐소생술을 시행해야 합니다."
  },
  {
    "question": "무맥성 심정지 상태의 소아 환자에게 심장압박을 시행할 때 분당 몇 회의 속도로 압박해야 하나요?",
    "reference_sentences": [
      "③ 최소 100회/분의 심장압박을 시행한다. 심장압박의 깊이는 영아에서는 최소 4cm, 소아에서는 최소 5cm가 되게 한다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 최소 100회/분의 심장압박을 시행한다 ##end_quote## 라고 되어 있습니다. 이는 심장압박을 분당 최소 100회의 속도로 시행해야 함을 의미합니다. <ANSWER>: 무맥성 심정지 상태의 소아 환자에게 심장압박을 시행할 때 분당 최소 100회의 속도로 압박해야 합니다."
  },
  {
    "question": "소아 환자에게 심장압박을 시행할 때 소아의 경우 압박 깊이는 최소 몇 cm가 되어야 하나요?",
    "reference_sentences": [
      "③ 최소 100회/분의 심장압박을 시행한다. 심장압박의 깊

QA 생성 중:  95%|█████████▌| 21/22 [04:52<00:14, 14.50s/it]

```json
[
  {
    "question": "2인 구조자 상황에서 소아 심폐소생술 시 심장압박과 환기 비율은 어떻게 되나요?",
    "reference_sentences": [
      "구조자인 경우 30번의 심장압박당 2회의 환기를, 2인 구조자인 경우 15번의 심장압박당 2회의 환기를 시행한다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 2인 구조자인 경우 15번의 심장압박당 2회의 환기를 시행한다 ##end_quote## 라고 명시되어 있습니다. 이는 소아 심폐소생술 시 2인 구조자 상황에서의 표준적인 비율입니다. <ANSWER>: 2인 구조자 상황에서 소아 심폐소생술 시 심장압박과 환기 비율은 15:2입니다."
  },
  {
    "question": "소아 심폐소생술 중 기관내삽관이 되어 있을 때 환기 속도는 어떻게 되나요?",
    "reference_sentences": [
      "기관내삽관이 되어 있다면 심장압박의 중단 없이 분당 10회의 환기를 시행한다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 기관내삽관이 되어 있다면 심장압박의 중단 없이 분당 10회의 환기를 시행한다 ##end_quote## 라고 되어 있습니다. 이는 기관내삽관 후 환기 속도에 대한 지침입니다. <ANSWER>: 소아 심폐소생술 중 기관내삽관이 되어 있을 때 환기 속도는 분당 10회입니다."
  },
  {
    "question": "소아 심폐소생술 시 심실세동에 대한 첫 제세동 에너지는 얼마인가요?",
    "reference_sentences": [
      "심실세동(ventricular fibrillation)/심실빈맥(pulseless ventricular tachycardia) 시 빠른 제세동이 필요하다. 제세동은 첫 시도에는 2 J/kg의 에너지로 시행한다."
    ],
    "answer": "##Rea

QA 생성 중: 100%|██████████| 22/22 [05:04<00:00, 13.83s/it]


✅ 생성 완료! 총 74개의 QA 생성됨.


In [121]:
print(f"[SKIPPED: answer format invalid] {question}")
print(f"[SKIPPED: reference mismatch] {question}")
print(f"[SKIPPED: trivial question] {question}")


[SKIPPED: answer format invalid] 소아 심폐소생술 중 심실세동이 반응하지 않을 때 두 번째 제세동 에너지는 얼마인가요?
[SKIPPED: reference mismatch] 소아 심폐소생술 중 심실세동이 반응하지 않을 때 두 번째 제세동 에너지는 얼마인가요?
[SKIPPED: trivial question] 소아 심폐소생술 중 심실세동이 반응하지 않을 때 두 번째 제세동 에너지는 얼마인가요?


In [122]:
import pandas as pd

# ✅ QA 리스트를 DataFrame으로 변환
rows = []
for item in qa_dataset:
    rows.append({
        "page": item.get("metadata", {}).get("page"),
        "question": item.get("question"),
        "reference_sentences": "\n".join(item.get("reference_sentences", [])),
        "answer": item.get("answer"),
    })

df = pd.DataFrame(rows)

# ✅ 엑셀로 저장
excel_output_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\raft_qa_dataset_35-56.xlsx"
df.to_excel(excel_output_path, index=False)

print(f"✅ 엑셀 파일 저장 완료: {excel_output_path}")


✅ 엑셀 파일 저장 완료: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\raft_qa_dataset_35-56.xlsx
